In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
import pandas as pd
import seaborn as sns

from river import compose
from river import preprocessing
from river import ensemble
from sklearn.model_selection import train_test_split

from source.config import *
from source.stability.incremental_stability_analyzer import IncrementalStabilityAnalyzer
from source.utils.incremental_model_utils import train_incremental_model
from source.data_loaders.folktables_dataset_from_pd import FolktablesDatasetFromPandas

In [4]:
sns.set(rc={'figure.figsize':(15, 5)})
dataset_name = 'folktables'
state = 'LA'
dataset_config = DATASETS_CONFIG[dataset_name]
TARGET_COLUMN = dataset_config['target_column']

In [5]:
# Creating a data frame
df = pd.read_csv(f"../../datasets/folktables-{state}-2018.csv", delimiter=',')
df.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,29,18,5,16,2,0,1,1,4,2,1,2,2,2,1,1,0
1,17,13,5,16,2,0,1,1,4,1,1,2,2,2,1,2,0
2,37,13,5,16,2,0,1,1,4,1,1,2,2,2,1,1,0
3,86,12,2,16,1,0,1,1,4,1,1,2,1,2,2,2,0
4,22,19,5,17,2,0,1,1,4,1,1,2,2,2,2,1,0


In [6]:
df.shape

(43588, 17)

In [7]:
df.columns

Index(['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR'],
      dtype='object')

In [8]:
df['ESR'].value_counts()

0    25363
1    18225
Name: ESR, dtype: int64

In [9]:
dataset_size = df.shape[0]
test_size = int(TEST_FRACTION * dataset_size)
validation_size = test_size

train_cv_df, test_df = train_test_split(df, test_size=test_size, random_state=SEED)
train_df, cv_df = train_test_split(train_cv_df, test_size=validation_size, random_state=SEED)
print("train_df shape: ", train_df.shape)
print("cv_df shape: ", cv_df.shape)
print("test_df shape: ", test_df.shape)

train_df shape:  (26154, 17)
cv_df shape:  (8717, 17)
test_df shape:  (8717, 17)


In [10]:
train_dataset = FolktablesDatasetFromPandas(pd_dataset=train_df)

In [11]:
numerical_features = dataset_config['numerical_features']
categorical_features = [feature for feature in df.columns if feature not in numerical_features + [TARGET_COLUMN]]

categorical_features

['MAR',
 'RELP',
 'DIS',
 'ESP',
 'CIT',
 'MIG',
 'MIL',
 'ANC',
 'NATIVITY',
 'DEAR',
 'DEYE',
 'DREM',
 'SEX',
 'RAC1P']

### Apply data transformations

In [12]:
def get_transform_pipeline(categorical_features, numerical_features):
    transform_pipeline = compose.Select(categorical_features[0])
    for feature in categorical_features[1:]:
        transform_pipeline += compose.Select(feature)
    transform_pipeline |= preprocessing.OneHotEncoder()

    for feature in numerical_features:
        transform_pipeline += compose.Select(feature)
    transform_pipeline |= preprocessing.MinMaxScaler()

    return transform_pipeline

In [13]:
transform_pipeline = get_transform_pipeline(categorical_features, numerical_features)

for idx, (x, y) in enumerate(train_dataset):
    print(f'Initial features: {x}\n')
    x = transform_pipeline.transform_one(x)
    print(f'Transformed features: {x}\n\n')
    if idx + 1 == 10:
        break

Initial features: {'AGEP': 93, 'SCHL': 19, 'MAR': '3', 'RELP': '0', 'DIS': '1', 'ESP': '0', 'CIT': '1', 'MIG': '1', 'MIL': '4', 'ANC': '4', 'NATIVITY': '1', 'DEAR': '2', 'DEYE': '1', 'DREM': '2', 'SEX': '2', 'RAC1P': '1'}

Transformed features: {'SCHL': 0.0, 'AGEP': 0.0, 'RAC1P_1': 0.0, 'SEX_2': 0.0, 'DREM_2': 0.0, 'DEYE_1': 0.0, 'DEAR_2': 0.0, 'NATIVITY_1': 0.0, 'ANC_4': 0.0, 'MIL_4': 0.0, 'MIG_1': 0.0, 'CIT_1': 0.0, 'ESP_0': 0.0, 'DIS_1': 0.0, 'RELP_0': 0.0, 'MAR_3': 0.0}


Initial features: {'AGEP': 58, 'SCHL': 23, 'MAR': '3', 'RELP': '0', 'DIS': '2', 'ESP': '0', 'CIT': '1', 'MIG': '1', 'MIL': '4', 'ANC': '2', 'NATIVITY': '1', 'DEAR': '2', 'DEYE': '2', 'DREM': '2', 'SEX': '1', 'RAC1P': '1'}

Transformed features: {'SCHL': 1.0, 'AGEP': 0.0, 'RAC1P_1': 0.0, 'SEX_2': 0.0, 'SEX_1': 0.0, 'DREM_2': 0.0, 'DEYE_2': 0.0, 'DEYE_1': 0.0, 'DEAR_2': 0.0, 'NATIVITY_1': 0.0, 'ANC_2': 0.0, 'ANC_4': 0.0, 'MIL_4': 0.0, 'MIG_1': 0.0, 'CIT_1': 0.0, 'ESP_0': 0.0, 'DIS_2': 0.0, 'DIS_1': 0.0, 'RELP_0': 0.

### Analyze stability of Adaptive Random Forest classifier

In [14]:
def get_base_model():
    return ensemble.AdaptiveRandomForestClassifier(n_models=10, seed=SEED, split_criterion='gini'), 'Adaptive Random Forest'

In [15]:
base_model, base_model_name = get_base_model()

# Conduct model training
base_model = train_incremental_model(base_model, train_dataset, train_df.shape[0])

2022-11-22 13:28:43 incremental_model_utils.py INFO    : Iteration 1000/26154 -- Accuracy: 75.08%; WeightedF1: 74.89%
2022-11-22 13:28:44 incremental_model_utils.py INFO    : Iteration 2000/26154 -- Accuracy: 77.29%; WeightedF1: 77.28%
2022-11-22 13:28:46 incremental_model_utils.py INFO    : Iteration 3000/26154 -- Accuracy: 77.83%; WeightedF1: 77.90%
2022-11-22 13:28:47 incremental_model_utils.py INFO    : Iteration 4000/26154 -- Accuracy: 78.37%; WeightedF1: 78.46%
2022-11-22 13:28:48 incremental_model_utils.py INFO    : Iteration 5000/26154 -- Accuracy: 78.82%; WeightedF1: 78.89%
2022-11-22 13:28:49 incremental_model_utils.py INFO    : Iteration 6000/26154 -- Accuracy: 78.73%; WeightedF1: 78.82%
2022-11-22 13:28:51 incremental_model_utils.py INFO    : Iteration 7000/26154 -- Accuracy: 78.93%; WeightedF1: 79.02%
2022-11-22 13:28:52 incremental_model_utils.py INFO    : Iteration 8000/26154 -- Accuracy: 79.07%; WeightedF1: 79.16%
2022-11-22 13:28:53 incremental_model_utils.py INFO    :



Metrics after incremental model training:
Accuracy: 80.03%
WeightedF1: 80.11%


In [16]:
stability_analyzer = IncrementalStabilityAnalyzer(base_model,
                                                  base_model_name=base_model_name,
                                                  train_pd_dataset=cv_df,
                                                  test_pd_dataset=test_df,
                                                  test_y_true=test_df[TARGET_COLUMN],
                                                  dataset_reader=FolktablesDatasetFromPandas,
                                                  dataset_name=dataset_name + '_LA',
                                                  n_estimators=200)

In [ ]:
stability_analyzer.measure_stability_metrics(make_plots=True)

2022-11-22 13:34:51 base_stability_analyzer.py INFO    : Start testing of classifier 1 / 200
2022-11-22 13:35:00 base_stability_analyzer.py INFO    : Classifier 1 / 200 was tested
2022-11-22 13:35:00 base_stability_analyzer.py INFO    : Start testing of classifier 2 / 200
2022-11-22 13:35:08 base_stability_analyzer.py INFO    : Classifier 2 / 200 was tested
2022-11-22 13:35:08 base_stability_analyzer.py INFO    : Start testing of classifier 3 / 200
2022-11-22 13:35:17 base_stability_analyzer.py INFO    : Classifier 3 / 200 was tested
2022-11-22 13:35:17 base_stability_analyzer.py INFO    : Start testing of classifier 4 / 200
2022-11-22 13:35:26 base_stability_analyzer.py INFO    : Classifier 4 / 200 was tested
2022-11-22 13:35:26 base_stability_analyzer.py INFO    : Start testing of classifier 5 / 200
2022-11-22 13:35:35 base_stability_analyzer.py INFO    : Classifier 5 / 200 was tested
2022-11-22 13:35:35 base_stability_analyzer.py INFO    : Start testing of classifier 6 / 200
2022-11